In [1]:
# import necessary libraries
import numpy as np 
import pandas as pd 
import librosa 
from glob import glob

In [2]:
#load dataset 
import os
data = pd.read_csv('extracted_chorus.csv')
df = data.copy()
df

,Artist,Title,Label,song_path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav
...,...,...,...,...,...
704,Rascal Flatts,Me And My Gang,0,unpopular/Me And My Gang.mp3,chorus_extract/Me And My Gang.wav
705,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav
706,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav
707,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav


In [3]:
from scipy.stats import skew, kurtosis  # import libs for statistics

In [4]:
paths = list(df['extracted_chorus_path']) # list for extracted chorus paths
titles = list(df['Title']) # list of song titles
data = [] # list to contain extracted audio features 

In [5]:
# helper function to get statistics of each dimension data for each audio raw feature 
def statistics(list, feature, columns_name, data):
    i = 0 # label to track the number of raw dimensions for each audio feature 
    for ele in list:
        # get the 7 statististics of each raw dimension
        _skew = skew(ele)
        columns_name.append(f'{feature}_kew_{i}')
        min = np.min(ele)
        columns_name.append(f'{feature}_min_{i}')
        max = np.max(ele)
        columns_name.append(f'{feature}_max_{i}')
        std = np.std(ele)
        columns_name.append(f'{feature}_std_{i}')
        mean = np.mean(ele)
        columns_name.append(f'{feature}_mean_{i}')
        median = np.median(ele)
        columns_name.append(f'{feature}_median_{i}')
        _kurtosis = kurtosis(ele)
        columns_name.append(f'{feature}_kurtosis_{i}')

        i += 1
        # append 7 statistics as single row in list 
        data.append(_skew) 
        data.append(min)
        data.append(max) 
        data.append(std) 
        data.append(mean) 
        data.append(median) 
        data.append(_kurtosis)
    
    return data

In [6]:
# helper function ectract audio feature with statistics 
def extract_features(audio_path, title):

    data = []
    columns_name = ['title']
    data.append(title)
    x , sr = librosa.load(audio_path) #load audio waveform and sample rate

    chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
    statistics(chroma_stft, 'chroma_stft', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
    statistics(chroma_cqt, 'chroma_cqt', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
    statistics(chroma_cens, 'chroma_cens', columns_name, data) # stats = 12 dims * 7 stats data

    mfcc = librosa.feature.mfcc(x, sr) # mfcc with 20 raw dims
    statistics(mfcc, 'mfcc', columns_name, data)  # stats = 20 dims * 7 stats data
    
    rms = librosa.feature.rms(x, sr) # rms with 1 raw dims
    statistics(rms, 'rms', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_centroid = librosa.feature.spectral_centroid(x , sr) # spectral_centroid with 1 raw dims
    statistics(spectral_centroid, 'spectral_centroid', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_bandwidth = librosa.feature.spectral_bandwidth(x , sr) # spectral_bandwidth with 1 raw dims
    statistics(spectral_bandwidth, 'spectral_bandwidth', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_contrast = librosa.feature.spectral_contrast(x , sr) # spectral_contrast with 7 raw dims
    statistics(spectral_contrast, 'spectral_contrast', columns_name, data) # stats = 7 dims * 7 stats data
    
    spectral_rolloff = librosa.feature.spectral_rolloff(x , sr) # spectral_rolloff with 1 raw dims
    statistics(spectral_rolloff, 'spectral_rolloff', columns_name, data) # stats = 1 dims * 7 stats data

    tonnetz = librosa.feature.tonnetz(x , sr) # tonnetz with 6 raw dims
    statistics(tonnetz, 'tonnetz', columns_name, data) # stats = 6 dims * 7 stats data

    zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr) # zero_crossing_rate with 1 raw dims
    statistics(zero_crossing_rate, 'zero_crossing_rate', columns_name, data) # stats = 1 dims * 7 stats data

    # total number of expected dimensions - 518

    return data, columns_name

In [7]:
# extract audio freatures for all songs chosuses/Hook
i = 0
while i < len(paths):
    audio_path = paths[i]
    d, cols = extract_features(audio_path, titles[i])
    data.append(d)
    print(f'The {i} song Done...')
    i += 1

C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:9: FutureWarning: Pass y=[0.33358765 0.3571167  0.23165894 ... 0.67459106 0.73965454 0.79852295], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:12: FutureWarning: Pass y=[0.33358765 0.3571167  0.23165894 ... 0.67459106 0.73965454 0.79852295], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:15: FutureWarning: Pass y=[0.33358765 0.3571167  0.23165894 ... 0.67459106 0.73965454 0.79852295], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_

The 0 song Done...


C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:12: FutureWarning: Pass y=[ 0.15301514  0.12756348  0.0765686  ... -0.23513794 -0.25631714
 -0.24804688], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:15: FutureWarning: Pass y=[ 0.15301514  0.12756348  0.0765686  ... -0.23513794 -0.25631714
 -0.24804688], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:18: FutureWarning: Pass y=[ 0.15301514  0.12756348  0.0765686  ... -0.23513794 -0.25631714
 -0.24804688], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.

The 1 song Done...


C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:9: FutureWarning: Pass y=[-0.11358643 -0.1204834  -0.13168335 ... -0.03881836 -0.03622437
 -0.03048706], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:12: FutureWarning: Pass y=[-0.11358643 -0.1204834  -0.13168335 ... -0.03881836 -0.03622437
 -0.03048706], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:15: FutureWarning: Pass y=[-0.11358643 -0.1204834  -0.13168335 ... -0.03881836 -0.03622437
 -0.03048706], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = li

...

The 706 song Done...


C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:9: FutureWarning: Pass y=[-0.05215454 -0.05322266 -0.04736328 ... -0.14880371 -0.15213013
 -0.14959717], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:12: FutureWarning: Pass y=[-0.05215454 -0.05322266 -0.04736328 ... -0.14880371 -0.15213013
 -0.14959717], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:15: FutureWarning: Pass y=[-0.05215454 -0.05322266 -0.04736328 ... -0.14880371 -0.15213013
 -0.14959717], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = li

The 707 song Done...


C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:9: FutureWarning: Pass y=[-0.00466919 -0.00448608 -0.00125122 ... -0.02374268 -0.00210571
  0.02618408], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:12: FutureWarning: Pass y=[-0.00466919 -0.00448608 -0.00125122 ... -0.02374268 -0.00210571
  0.02618408], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
C:\Users\kingr\AppData\Local\Temp/ipykernel_4756/404159168.py:15: FutureWarning: Pass y=[-0.00466919 -0.00448608 -0.00125122 ... -0.02374268 -0.00210571
  0.02618408], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = li

The 708 song Done...


In [9]:
newData = pd.DataFrame(data, columns=cols) # create dataframe for new extracted audio features 
newData

,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,chroma_stft_mean_0,chroma_stft_median_0,chroma_stft_kurtosis_0,chroma_stft_kew_1,chroma_stft_min_1,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,Blinding Lights,0.475515,0.016034,1.0,0.290181,0.473255,0.422056,-0.921490,0.742849,0.015807,...,0.007882,0.001048,-0.539834,1.466814,0.037551,0.159229,0.020931,0.082646,0.079751,3.242503
1,Good 4 U,1.048792,0.006909,1.0,0.274429,0.347852,0.255082,0.142051,0.299729,0.004950,...,-0.010268,-0.011481,0.137893,-0.398137,0.054286,0.224626,0.035255,0.147474,0.157392,-0.313862
2,Drivers License,1.401351,0.000672,1.0,0.308941,0.235789,0.084969,0.741018,1.673569,0.000572,...,-0.001788,-0.011075,0.471367,0.796360,0.029388,0.166259,0.029590,0.073254,0.063628,-0.102664
3,Montero (Call Me By Your Name),0.896736,0.002983,1.0,0.239848,0.300507,0.263668,0.229533,0.847104,0.003551,...,-0.014387,-0.022632,0.471221,0.528719,0.048934,0.266757,0.043461,0.143693,0.137687,-0.347516
4,Butter,0.295846,0.036609,1.0,0.334841,0.522315,0.452251,-1.436077,0.838113,0.049554,...,0.006773,0.004753,-0.391601,2.536822,0.040181,0.221769,0.027223,0.095766,0.090703,7.626171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,Me And My Gang,0.625285,0.027336,1.0,0.277975,0.438840,0.384980,-0.596620,0.916433,0.015900,...,-0.005838,-0.006652,0.096056,0.380751,0.076599,0.211837,0.026225,0.145236,0.141247,-0.142441
705,Crash,0.249745,0.023226,1.0,0.311153,0.497409,0.444879,-1.332315,-0.236763,0.033732,...,-0.029686,-0.026330,0.825543,1.568743,0.058367,0.322585,0.043109,0.152655,0.140748,2.920797
706,Luxurious,-0.040181,0.015263,1.0,0.327108,0.575447,0.557280,-1.401559,1.204261,0.014560,...,0.003071,0.007935,0.262104,1.681439,0.044308,0.400544,0.067331,0.151410,0.135760,3.043618
707,Unbreakable,0.414345,0.029603,1.0,0.288675,0.484886,0.404017,-1.174093,0.671072,0.017485,...,0.017034,0.002137,0.878099,0.796395,0.010340,0.066531,0.012039,0.029463,0.028481,0.431357


In [10]:
df #previous data table

,Artist,Title,Label,song_path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav
...,...,...,...,...,...
704,Rascal Flatts,Me And My Gang,0,unpopular/Me And My Gang.mp3,chorus_extract/Me And My Gang.wav
705,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav
706,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav
707,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav


In [11]:
final_data = df.join(newData) # join both datasets of same number of rows 
final_data

,Artist,Title,Label,song_path,extracted_chorus_path,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav,Blinding Lights,0.475515,0.016034,1.0,0.290181,...,0.007882,0.001048,-0.539834,1.466814,0.037551,0.159229,0.020931,0.082646,0.079751,3.242503
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav,Good 4 U,1.048792,0.006909,1.0,0.274429,...,-0.010268,-0.011481,0.137893,-0.398137,0.054286,0.224626,0.035255,0.147474,0.157392,-0.313862
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav,Drivers License,1.401351,0.000672,1.0,0.308941,...,-0.001788,-0.011075,0.471367,0.796360,0.029388,0.166259,0.029590,0.073254,0.063628,-0.102664
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav,Montero (Call Me By Your Name),0.896736,0.002983,1.0,0.239848,...,-0.014387,-0.022632,0.471221,0.528719,0.048934,0.266757,0.043461,0.143693,0.137687,-0.347516
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav,Butter,0.295846,0.036609,1.0,0.334841,...,0.006773,0.004753,-0.391601,2.536822,0.040181,0.221769,0.027223,0.095766,0.090703,7.626171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,Rascal Flatts,Me And My Gang,0,unpopular/Me And My Gang.mp3,chorus_extract/Me And My Gang.wav,Me And My Gang,0.625285,0.027336,1.0,0.277975,...,-0.005838,-0.006652,0.096056,0.380751,0.076599,0.211837,0.026225,0.145236,0.141247,-0.142441
705,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav,Crash,0.249745,0.023226,1.0,0.311153,...,-0.029686,-0.026330,0.825543,1.568743,0.058367,0.322585,0.043109,0.152655,0.140748,2.920797
706,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav,Luxurious,-0.040181,0.015263,1.0,0.327108,...,0.003071,0.007935,0.262104,1.681439,0.044308,0.400544,0.067331,0.151410,0.135760,3.043618
707,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav,Unbreakable,0.414345,0.029603,1.0,0.288675,...,0.017034,0.002137,0.878099,0.796395,0.010340,0.066531,0.012039,0.029463,0.028481,0.431357


In [12]:
final_data.columns # scan for repeated column names 

Index(['Artist', 'Title', 'Label', 'song_path', 'extracted_chorus_path',
       'title', 'chroma_stft_kew_0', 'chroma_stft_min_0', 'chroma_stft_max_0',
       'chroma_stft_std_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=524)

In [13]:
final_data = final_data.drop(columns=['title'], axis=1) #drop repeated colunm name

In [14]:
final_data.columns

Index(['Artist', 'Title', 'Label', 'song_path', 'extracted_chorus_path',
       'chroma_stft_kew_0', 'chroma_stft_min_0', 'chroma_stft_max_0',
       'chroma_stft_std_0', 'chroma_stft_mean_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=523)

In [15]:
#save table to csv file 
final_data.to_csv('feature_extract.csv', index=False) 